### KPIs

#### Import Zone

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from functools import reduce

In [2]:
df_final_experiment_clients = pd.read_csv('C:/Users/user/Jupyter/IronHack/Work_Python/Week5/MiniProject/df_final_experiment_clients.csv',sep=';')

In [3]:
df_concat_final_web_data = pd.read_csv('C:/Users/user/Jupyter/IronHack/Work_Python/Week5/MiniProject/df_concat_final_web_data.csv',sep=';')

In [4]:
df_final_demo = pd.read_csv('C:/Users/user/Jupyter/IronHack/Work_Python/Week5/MiniProject/df_final_demo.csv',sep=';')

#### Merge of df_final_experiment_clients & df_concat_final_web_data

In [5]:
# Lista de DataFrames
interactions_and_testA_B_df = [df_final_experiment_clients,df_concat_final_web_data]

# Función para unir de forma acumulativa
interactions_df = reduce(lambda left, right: pd.merge(left, right, on='client_id', how='left'), interactions_and_testA_B_df)
# interactions_df.drop(columns=['Unnamed: 0_x'], inplace=True)
# interactions_df.drop(columns=['Unnamed: 0_y'], inplace=True)
interactions_df

,client_id,Variation,visitor_id,visit_id,process_step,date_time
0,9988021,Test,580560515_7732621733,781255054_21935453173_531117,3_step_3,2017-04-17 15:27:07
1,9988021,Test,580560515_7732621733,781255054_21935453173_531117,2_step_2,2017-04-17 15:26:51
2,9988021,Test,580560515_7732621733,781255054_21935453173_531117,3_step_3,2017-04-17 15:19:22
3,9988021,Test,580560515_7732621733,781255054_21935453173_531117,2_step_2,2017-04-17 15:19:13
4,9988021,Test,580560515_7732621733,781255054_21935453173_531117,3_step_3,2017-04-17 15:18:04
...,...,...,...,...,...,...
321304,6334360,Test,629124187_65258232847,586047816_14599436811_389185,2_step_2,2017-04-11 08:45:38
321305,6334360,Test,629124187_65258232847,586047816_14599436811_389185,1_step_1,2017-04-11 08:44:43
321306,6334360,Test,629124187_65258232847,586047816_14599436811_389185,2_step_2,2017-04-11 08:44:30
321307,6334360,Test,629124187_65258232847,586047816_14599436811_389185,1_step_1,2017-04-11 08:42:36


#### Completion Rate: The proportion of users who reach the final ‘confirm’ step.

In [6]:
# Groupby process step by Control 
filtered_df_3 = interactions_df[(interactions_df['Variation'] == 'Control') & (interactions_df['process_step'] == '0_start')]
completion_rate__control_denominator = filtered_df_3.groupby(['Variation', 'process_step'])['client_id'].nunique()
filtered_df_4 = interactions_df[(interactions_df['Variation'] == 'Control') & (interactions_df['process_step'] == '4_confirm')]
completion_rate_control_numerator = filtered_df_4.groupby(['Variation', 'process_step'])['client_id'].nunique()

In [7]:
# Groupby process step by Test 
filtered_df_1 = interactions_df[(interactions_df['Variation'] == 'Test') & (interactions_df['process_step'] == '0_start')]
completion_rate_test_denominator = filtered_df_1.groupby(['Variation', 'process_step'])['client_id'].nunique()
filtered_df_2 = interactions_df[(interactions_df['Variation'] == 'Test') & (interactions_df['process_step'] == '4_confirm')]
completion_rate_test_numerator = filtered_df_2.groupby(['Variation', 'process_step'])['client_id'].nunique()

In [8]:
completion_rate_control_percentage = np.round(((completion_rate_control_numerator.values / completion_rate__control_denominator.values) * 100), 2)

print(f'Completion rate for Control group is: {completion_rate_control_percentage}')

Completion rate for Control group is: [65.97]


In [9]:
completion_rate_test_percentage = np.round(((completion_rate_test_numerator.values / completion_rate_test_denominator.values) * 100), 2)

print(f'Completion rate for Test group is: {completion_rate_test_percentage}')

Completion rate for Test group is: [70.04]


#### Time Spent on Each Step: The average duration users spend on each step.

In [10]:
# Convert 'date_time' to datetime format
interactions_df['date_time'] = pd.to_datetime(interactions_df['date_time'])

# Sort DataFrame by 'client_id' and 'date_time'
interactions_df = interactions_df.sort_values(by=['client_id', 'date_time'])

# Calculate time spent on each step
interactions_df['time_spent'] = interactions_df.groupby('client_id')['date_time'].diff().dt.total_seconds()



In [11]:
average_time_per_step_control = interactions_df[interactions_df['Variation'] == 'Control'].groupby('process_step')['time_spent'].mean().round(2)
average_time_per_step_control

process_step
0_start      367064.54
1_step_1       1838.78
2_step_2         39.34
3_step_3        575.22
4_confirm     66077.31
Name: time_spent, dtype: float64

In [12]:
percentages_control = round(((average_time_per_step_control / average_time_per_step_control.sum()) * 100),)
percentages_control

process_step
0_start      84.0
1_step_1      0.0
2_step_2      0.0
3_step_3      0.0
4_confirm    15.0
Name: time_spent, dtype: float64

In [13]:
average_time_per_step_test = interactions_df[interactions_df['Variation'] == 'Test'].groupby('process_step')['time_spent'].mean().round(2)
average_time_per_step_test

process_step
0_start      269749.87
1_step_1       1339.80
2_step_2        111.21
3_step_3        738.92
4_confirm    104432.78
Name: time_spent, dtype: float64

In [14]:
percentages_test = round(((average_time_per_step_test / average_time_per_step_test.sum()) * 100),2)
percentages_test

process_step
0_start      71.67
1_step_1      0.36
2_step_2      0.03
3_step_3      0.20
4_confirm    27.75
Name: time_spent, dtype: float64

#### Error Rates: If there’s a step where users go back to a previous step, it may indicate confusion or an error. You should consider moving from a later step to an earlier one as an error.

In [16]:
# Convert 'date_time' to datetime format
interactions_df['date_time'] = pd.to_datetime(interactions_df['date_time'])

# Sort DataFrame by 'client_id' and 'date_time'
interactions_df = interactions_df.sort_values(by=['client_id','visit_id', 'date_time'])

# Identify cases where users go back to a previous step
interactions_df['previous_step'] = interactions_df.groupby('client_id')['process_step'].shift(1)
interactions_df['is_error'] = interactions_df['process_step'] <= interactions_df['previous_step']
interactions_df['is_error_count'] = interactions_df['is_error'].astype(int)
interactions_df.to_csv('C:/Users/user/Jupyter/IronHack/Work_Python/Week5/MiniProject/interactions_df.csv',index=False,encoding='latin-1',sep=';')


In [47]:
# Count the number of errors
error_count_control = interactions_df[interactions_df['Variation'] == 'Control']['is_error'].sum()

# Calculate the total number of transitions between steps
# Total rows - number of unique client IDs
# total_transitions_control = len(interactions_df['Variation'] == 'Control') - interactions_df[interactions_df['Variation'] == 'Control']['client_id'].nunique()
total_transitions_control_new = (interactions_df[interactions_df['Variation'] == 'Control']['client_id'].nunique())*5
# Calculate the error rate
error_rate_control = round(((error_count_control / total_transitions_control_new)*100),2)

print("Error rate for Control group:", error_rate_control, error_count_control, total_transitions_control_new)

Error rate for Control group: 29.34 34517 117660


In [49]:
# Count the number of errors
error_count_Test = interactions_df[interactions_df['Variation'] == 'Test']['is_error'].sum()

# Calculate the total number of transitions between steps
# Total rows - number of unique client IDs
# total_transitions_Test = len(interactions_df['Variation'] == 'Test') - interactions_df[interactions_df['Variation'] == 'Test']['client_id'].nunique()
total_transitions_Test_new = (interactions_df[interactions_df['Variation'] == 'Test']['client_id'].nunique())*5
# Calculate the error rate
error_rate_Test = round(((error_count_Test / total_transitions_Test_new)*100),2)

print("Error rate for Test group:", error_rate_Test, error_count_Test, total_transitions_Test_new)

Error rate for Test group: 34.13 46015 134840
